Loading the dataset (output of week_1)

In [0]:
df = spark.read.table('week_1_output_2_csv')
df.show()

+----------+---------------+-----------------+------------------+--------+-------------+---------------+----------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+------------------+
|start_time|start time hour|start time minute|start time seconds|end_time|end_time hour|end_time minute|end_time seconds|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|member_birth_year|member_gender|     pyment|haversine_distance|
+----------+---------------+-----------------+------------------+--------+-------------+---------------+----------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+-

Task 1: Calculate the duration in seconds of each trip

In [0]:
import pyspark.sql.functions as F

In [0]:
# making timestamp columns for start and end times just for easier and cleaner calculations
df = df.withColumn('start_time_timestamp', F.to_timestamp(F.concat_ws(':', 'start time hour', 'start time minute', 'start time seconds')))
df = df.withColumn('end_time_timestamp', F.to_timestamp(F.concat_ws(':', 'end_time hour', 'end_time minute', 'end_time seconds')))
# NOTICE: to_timestamp function will create timestamp in format yyyy:MM:dd HH:mm:ss and year, month, day part will be the current date which is not a problem since our calculations will use only hh:mm:ss part.

In [0]:
# cast('long') transforms timestamp into the number of seconds.
df = df.withColumn('trip_duration[seconds]', F.col('end_time_timestamp').cast('long') - F.col('start_time_timestamp').cast('long'))

Task 2: By assuming each minute cost 0.35 cent calculate the fee for each trip

In [0]:
df = df.withColumn('trip_fee[$]', F.round(F.col('trip_duration[seconds]')/60*0.35, 2))

Task 3: Calculate the total distance for each bike and list the top 10

In [0]:
df.groupBy("bike_id").agg(F.sum("haversine_distance").alias("sum_distance")).sort(F.desc("sum_distance")).show(10)

+-------+-----------------+
|bike_id|     sum_distance|
+-------+-----------------+
|     68|743.3694820484666|
|   2178|721.1822432213564|
|    256|671.9160978829856|
|    235|670.1620031683864|
|   2049|656.8281017664259|
|    441|656.6422746252359|
|   2226|647.8232304063756|
|    796|646.8677297229269|
|    190|640.2941817412235|
|   2365|639.4126742864071|
+-------+-----------------+
only showing top 10 rows

